Determining MAX_SAD for Segmentation Model (based on synthetic noise level)


In [ ]:

testing_regions = ["b", "l", "o", "i", "p"]
m_type_map = { 0:"b", 1:"l", 2 :"o", 3 :"i", 4:"p"}
region_ms = {"b": [0.3, 0.2,.1,0.4],
                "l": [0.7, 0, 0.3,0],
                "o": [0, 0.6,0.4,0],
                "i": [0.8, 0.2, 0, 0],
                "p": [0.3, 0, 0, 0.7]}
region_Ds = {"b": [200, 300, 100, 200],
                "l": [200, 300, 100, 200],
                "o": [200, 300, 100, 200],
                "i": [200, 300, 100, 200],
                "p": [200, 300, 100, 200]}

#  finding max sad
# If their SAD is <MAX_SAD we merge them. 
# Comes from manually examining pure data spectra vs. derived Hapke model spectra where dominant mineral is 80%. 
#
from model.segmentation import get_SAD

m_map = {}
D_map = {}
ttype='b'
for index, endmember in enumerate(USGS_PURE_ENDMEMBERS):
    m_map[endmember] = region_ms[ttype][index]
    D_map[endmember] = region_Ds[ttype][index]
r = get_USGS_r_mixed_hapke_estimate(m_map, D_map)
    
with open(R_DIR + "../wavelengths.pickle", 'rb') as handle:
    wavelengths = pickle.load(handle)

NOISE_AMOUNT=0
sads=[]
for t in range(50):
    noise = np.random.normal(loc=0, 
                             scale=NOISE_AMOUNT, 
                             size=len(wavelengths)) 

    noisy_r = r + noise
    for i, v in enumerate(noisy_r):
        if v <= 0:
            noisy_r[i]=0

    s=get_SAD(a=r, b=noisy_r)
    sads.append(s)
avg_SAD = np.mean(s)
print("Average SAD between pure and noisy R for ttype " + str(ttype) + 
      " with noise = " + str(NOISE_AMOUNT) + " is = " + str(round(avg_SAD,9)))
# print("we merge when target SAD < MAX_SAD, so wouldthis pass? " + str(avg_SAD<MAX_SAD))

fig, ax = plt.subplots(figsize=(6, 5), dpi=400)
ax.plot(wavelengths, r, color="blue", label="Orig")
ax.plot(wavelengths, noisy_r, color="red", label="Noisy")
ax.set_ylabel("Reflectance")
ax.set_xlabel("Wavelength")
ax.set_ylim((0, 1))
ax.set_xlim((min(wavelengths), max(wavelengths)))
plt.legend(loc='best', fontsize=12)
plt.show()

